# Creating a Blank Jupyter Notebook

In [ ]:
# ---------- Simple Sample Image Generation ----------
prompt = "An older female librarian"
num_steps = 4  # Turbo supports 1-4 steps
seed = 42

# Generate with turbo
generator = torch.Generator(device=device).manual_seed(seed)
sample_image = pipe_std_turbo(
    prompt,
    guidance_scale=0.0,  # ADD-style for Turbo
    num_inference_steps=num_steps,
    generator=generator,
    width=512, height=512,
).images[0]

print(f"Generated image: {sample_image.size}")
sample_image

In [ ]:
import json
import os
from pathlib import Path
from tqdm import tqdm

# ---------- Load prompts from JSON file ----------
prompts_file = "/home/azureuser/cloudfiles/code/Users/Andrew.Revell/nag-schedule/data/prompts_noun_negative.json"
output_dir = "/home/azureuser/cloudfiles/code/Users/Andrew.Revell/nag-schedule/results"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load prompts
with open(prompts_file, 'r') as f:
    prompts_data = json.load(f)

print(f"Loaded {len(prompts_data)} prompts from {prompts_file}")
print(f"Output directory: {output_dir}")

# ---------- Generate and save images ----------
num_steps = 4  # Turbo uses 4 steps
generated_count = 0

for idx, item in enumerate(tqdm(prompts_data)):
    prompt = item["prompt"]
    negative_prompt = item["negative_prompt"]
    seeds = item.get("seeds", [42])
    
    # Use first seed for quick generation
    seed = seeds[0]
    
    try:
        # Generate image
        generator = torch.Generator(device=device).manual_seed(seed)
        image = pipe_std_turbo(
            prompt,
            guidance_scale=0.0,
            num_inference_steps=num_steps,
            generator=generator,
            width=512, height=512,
        ).images[0]
        
        # Create output filename
        group = item.get("group", "unknown")
        filename = f"{idx:04d}_{group}_{seed}.png"
        filepath = os.path.join(output_dir, filename)
        
        # Save image
        image.save(filepath)
        generated_count += 1
        
    except Exception as e:
        print(f"Error generating image for prompt {idx}: {e}")
        continue

print(f"\n✓ Generated and saved {generated_count} images to {output_dir}")